In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('/Users/daniel/Princeton Dropbox/Daniel Gurevich/Research/discrete_sr/code/SPIDER_discrete')

import numpy as np
from sparse_reg_bf import *
from sparse_reg import regress

In [2]:
w = 100
h = 150
v = np.zeros([w, 1])
k = 10
#v[0:10, 0] = [1, 1, 1, 1, 1, 1, 0.8, 0.6, 0.4, 0.2]
v[0:10, 0] = np.array(range(10, 0, -1))/10

def generate_random_Q(inv_snr):
    # inv_snr: (signal to noise ratio)^-1
    U = np.random.uniform(-0.5, 0.5, [h, w])
    W = np.random.uniform(0, 1, [h, w])
    #U = np.random.uniform(0, 1, [w, h])
    
    #Sigma = (U.T @ U)/p + snr/(np.linalg.norm(v)**2)*(v @ v.T)
    
    true_matrix = W - (W @ v @ v.T)/(np.linalg.norm(v)**2)
    inv_row_norms = np.reshape(1/np.linalg.norm(true_matrix, axis=1), [h, 1])
    true_matrix = true_matrix * inv_row_norms # rescale row norms to 1
    Q = true_matrix+U*inv_snr/np.sqrt(w)
    return Q

def assess_Q(Q):
    true_residual = np.linalg.norm(Q @ v)
    _, _, V = np.linalg.svd(Q)
    best_sv = V[-1, :]/V[-1, 0]
    return true_residual, best_sv

#Q = generate_random_Q(sigma)
#true_residual, best_sv = assess_Q(Q)
#print("Q:", Q)
#print("Residual with true solution:", true_residual)
#print("Best singular vector:", best_sv)

In [23]:
#%%prun

sigma = 1/3 # (signal to noise ratio)^-1
N_trials = 100

verbose = False

np.random.seed(1)

pr_vec = []
sr_vec = []
for i in range(N_trials):
    print("i: ", i)
    scaler = Scaler(sub_inds=None, char_sizes=np.ones([w]), row_norms=None, unit_rows=True)
    #init = Initializer(method='combinatorial', start_k=2)
    #init = Initializer(method='combinatorial', start_k=9999)
    init = Initializer(method='power', start_k=20)
    res = Residual(residual_type='matrix_relative')
    iterator = ModelIterator(max_k=20, backward_forward=True, max_passes=0)
    thres = Threshold(threshold_type=None, n_terms=10)

    Q = generate_random_Q(sigma)
    true_residual, best_sv = assess_Q(Q)
    reg_result = sparse_reg_bf(Q, scaler, init, res, iterator, thres, verbose=verbose)
    xi2, _ = regress(Q, list(range(k)), np.ones([w]))
    
    achieved_residual = np.linalg.norm(Q @ reg_result.xi)
    adj_true_residual = np.linalg.norm(Q @ xi2)
    performance_ratio = achieved_residual/adj_true_residual
    support_recovery = sum(abs(reg_result.xi[0:k])>0)/k
    pr_vec.append(performance_ratio)
    sr_vec.append(support_recovery)
    #print("Recovered coefficient vector:", reg_result.xi)
    #print("Recovered support:", np.where(abs(reg_result.xi>0)))
    #print("Ratio of achieved residual to residual achieved by true solution: ", performance_ratio)
    #print("Fraction of correct support recovered: ", support_recovery)

print("Mean performance ratio:", np.round(np.mean(pr_vec), 3))
print("Mean support recovery:", np.round(np.mean(sr_vec), 3))

i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
Mean performance ratio: 1.06
Mean support recovery: 0.905


In [4]:
print(best_sv)
print(reg_result.xi)

[ 1.000e+00  9.479e-01  7.500e-01  6.780e-01  5.514e-01  4.933e-01
  4.058e-01  3.209e-01  1.850e-01  1.016e-01 -3.125e-02  2.196e-04
  9.260e-02  4.262e-02 -3.650e-02  1.985e-02 -1.441e-02 -6.097e-02
  1.029e-01 -8.667e-02 -8.671e-02  1.846e-02  4.169e-02  8.849e-02
 -4.341e-03 -1.487e-01 -1.410e-02  4.575e-03  3.624e-02 -7.670e-02
  5.422e-03 -3.686e-02 -4.298e-02  4.508e-02 -3.775e-02  1.342e-02
  8.960e-03  2.111e-02  1.769e-03 -4.995e-02  8.683e-03 -7.696e-02
 -7.387e-02 -4.358e-02  8.461e-02 -2.642e-02 -4.957e-02  6.412e-02
  8.419e-02  4.924e-03 -7.768e-02 -1.162e-02  3.654e-02 -7.564e-02
 -1.868e-02  1.068e-01  1.163e-02  1.989e-02 -2.390e-02 -6.328e-02
  1.615e-02 -5.153e-03  3.501e-02  3.027e-02 -4.928e-02 -8.570e-02
  1.841e-02  4.848e-02 -1.624e-02  1.082e-01  5.810e-02 -3.736e-02
 -5.073e-02 -3.083e-02  1.377e-02  5.349e-02 -1.213e-02  1.482e-01
  5.721e-02  4.465e-02  3.618e-02  1.185e-02  2.723e-03 -7.372e-02
  6.345e-02 -2.145e-02  6.549e-02  5.158e-02 -1.210e-01 -6.774

In [5]:
SNR = 3, v2

combinatorial k=2, no bf (=1 bf):
Mean performance ratio: 1.299
Mean support recovery: 0.811

(k=3:
Mean performance ratio: 1.06
Mean support recovery: 0.905)

..., 2 bf:
Mean performance ratio: 1.194
Mean support recovery: 0.888

..., 3 bf:
Mean performance ratio: 1.202
Mean support recovery: 0.859

..., 4 bf:
Mean performance ratio: 1.183
Mean support recovery: 0.894

..., 5 bf:
Mean performance ratio: 1.202
Mean support recovery: 0.859

[...]

..., 10 bf:
Mean performance ratio: 1.183
Mean support recovery: 0.894

---

combinatorial k=9999, no bf:
Mean performance ratio: 0.999
Mean support recovery: 0.962

---

power (k=10), no bf:
Mean performance ratio: 1.068
Mean support recovery: 0.901

power (k=15), no bf:
Mean performance ratio: 1.014
Mean support recovery: 0.934

power (k=20), no bf:
Mean performance ratio: 1.005
Mean support recovery: 0.949

power (k=20), more bf: (no better)

power (k=30), no bf:
Mean performance ratio: 1.002
Mean support recovery: 0.957

power (k=40), no bf:
Mean performance ratio: 1.0
Mean support recovery: 0.961

SyntaxError: invalid syntax (1520841938.py, line 3)

In [ ]:
SNR = 3

start_k=9999, no bf: (245 sec)
Mean performance ratio: 1.001
Mean support recovery: 0.994

start_k=2, no bf: (22 sec)
Mean performance ratio: 1.305
Mean support recovery: 0.815

start_k=2, 2 bf passes:
Mean performance ratio: 1.266
Mean support recovery: 0.865

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.158
Mean support recovery: 0.895

power (k=20), no bf: (3 sec)
Mean performance ratio: 2.078
Mean support recovery: 0.567

power (k=20), 10 bf passes: (55 sec)
Mean performance ratio: 1.076
Mean support recovery: 0.946

In [ ]:
SNR = 2 (most PRs compared to the true solution, not adjusted solution on true support!)

start_k=9999, no bf: (255 sec)
Mean performance ratio: 0.892 / 0.891 (actually 0.997)
Mean support recovery: 0.954 / 0.971

start_k=2, no bf:
Mean performance ratio: 1.072
Mean support recovery: 0.74

start_k=2, 2 bf passes:
Mean performance ratio: 1.065
Mean support recovery: 0.75 / 0.793

start_k=2, 10 bf passes: (65 sec)
Mean performance ratio: 1.009 / 0.983
Mean support recovery: 0.847 / 0.841
 
power (k=20), no bf:
Mean performance ratio: 1.684
Mean support recovery: 0.328

power (k=40), no bf:
Mean performance ratio: 1.651
Mean support recovery: 0.319

power (k=20), 2 bf passes:
Mean performance ratio: 1.118
Mean support recovery: 0.723

power (k=20), 10 bf passes: (56 sec)
Mean performance ratio: 0.944 / 0.977
Mean support recovery: 0.898 / 0.848

power (k=20), 50 bf passes: (263 sec)
Mean performance ratio: 0.975
Mean support recovery: 0.864

power (k=40), 10 bf passes: (226 sec)
Mean performance ratio: 0.963
Mean support recovery: 0.857